# Projeto Final do Curso de NLP do I2A2

## Name Entity Recognition - NER

In [26]:
#libraries basics
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import json

import spacy 
from spacy.util import minibatch, compounding
from spacy import displacy
nlp_spacy = spacy.load('pt_core_news_md')

In [27]:
nlp_spacy.pipe_names
ner=nlp_spacy.get_pipe("ner")


In [28]:
PATH = '../data'
train = pd.read_json(PATH+'/train.json', lines=True)
valid = pd.read_json(PATH+'/dev.json', lines=True)

## Base Exploration

In [29]:
train.head()

,id,text,meta,annotation_approver,labels
0,1346,Regulamento REGULAMENTO DO ANTARES FUNDO DE IN...,{},NaN,"[[5117, 5121, percentual_credito], [27, 69, no..."
1,1347,"São Paulo, 12 de novembro de 2001 REGULAMENTO ...",{},NaN,"[[3782, 3786, alavancado], [49, 136, nome], [1..."
2,1348,DO FUNDO REGULAMENTO DO FUNDO DE INVESTIMENTO ...,{},NaN,"[[19672, 19677, perfil_tx_adm], [19798, 19810,..."
3,1349,MINUTA DO REGULAMENTO MASTER marcado BANRISUL ...,{},NaN,"[[126, 202, nome], [37, 113, nome], [6834, 684..."
4,1350,1 Ouvidoria JPMorgan: Tel.: 0800 – 7700847 / E...,{},NaN,"[[23384, 23395, tributação], [33163, 33171, pe..."


In [30]:
train.tail()

,id,text,meta,annotation_approver,labels
146,1534,REGULAMENTO DO TREND SHORT DOLAR FUNDO DE INVE...,{},NaN,"[[5042, 5045, percentual_credito], [5924, 5930..."
147,1535,REGULAMENTO DO THOA-THOA FUNDO DE INVESTIMENTO...,{},NaN,"[[18125, 18130, perfil_tx_adm], [5243, 5254, p..."
148,1536,REGULAMENTO DO TG BLUE MASTER FUNDO DE INVESTI...,{},NaN,"[[5324, 5327, percentual_credito], [6207, 6213..."
149,1537,REGULAMENTO DO CLARITAS QUANT MASTER FUNDO DE ...,{},NaN,"[[5073, 5076, percentual_credito], [5996, 6003..."
150,1443,Minuta Regulamento_BB ATACADO FEDERAL FI RENDA...,{},NaN,"[[484, 497, duração], [157, 175, cnpj_fundo], ..."


In [31]:
valid.head()

,id,text,meta,annotation_approver,labels
0,1457,REGULAMENTO FUNDOS APLIC AUT 1 REGULAMENTO DO ...,{},NaN,"[[20046, 20057, tributação], [36875, 36885, al..."
1,1458,2016-06-16 (1) MICROFILMADO I 000 ;~-;; 990 I ...,{},NaN,"[[9312, 9323, alavancado], [9905, 9916, percen..."
2,1459,Regulamento REGULAMENTO DO PACIFICO AZUL FUNDO...,{},NaN,"[[5851, 5854, percentual_credito], [6744, 6750..."
3,1460,CAPÍTULO I – DO FUNDO REGULAMENTO DO TALER DES...,{},NaN,"[[2365, 2376, tributação], [544, 557, duração]..."
4,1461,282 - Regul BB Previdenciário RF TP IPCA I - ...,{},NaN,"[[495, 508, duração], [172, 190, cnpj_fundo], ..."


In [32]:
valid.tail()

,id,text,meta,annotation_approver,labels
15,1472,Regulamento REGULAMENTO DO ALOCAÇÃO FUNDO DE I...,{},NaN,"[[4527, 4531, percentual_credito], [5483, 5490..."
16,1473,1 O presente Regulamento é parte integrante da...,{},NaN,"[[10443, 10454, tributação], [13450, 13491, Ge..."
17,1474,Regulamento 2181962v1 - Página 1 de 19- CARTE...,{},NaN,"[[20273, 20279, alavancado], [6617, 6620, perc..."
18,1476,REGULAMENTO WESTERN ASSET ADVANTAGE IPCA FUNDO...,{},NaN,"[[7351, 7357, alavancado], [393, 406, duração]..."
19,1477,Ethan Frome 1 de 13 Contatos do Administrador ...,{},NaN,"[[46509, 46520, tributação], [438, 451, duraçã..."


In [33]:
train.shape

(151, 5)

In [34]:
valid.shape

(20, 5)

In [35]:
train.isnull().sum()

id                       0
text                     0
meta                     0
annotation_approver    151
labels                   0
dtype: int64

## Spacy - NER 

In [36]:
#vamos converter os arquivos para o formato que o spacy aceita 
def convert_format_spacy():
    TRAIN_DATA =[]
    text = []
    with open ('../data/train.json', 'r',  encoding='utf8') as f:
        data = f.readlines()
    for word_in_sentence in data:
        word_in_sentence = json.loads(word_in_sentence)
        text.append(word_in_sentence)
    for i in text:
        if['id'] not in (1436,1520):
            entity = (i['text'], {'entities': i['labels']})
            TRAIN_DATA.append(entity)
    return TRAIN_DATA

In [37]:
TRAIN_DATA = convert_format_spacy()

In [38]:
entidades = ['percentual_credito',
 'nome',
 'cnpj_fundo',
 'condominio_aberto',
 'duracao',
 'composicao_n',
 'tipo_investidor',
 'administrador',
 'Gestor',
 'Custodiante',
 'investimento_ext',
 'tx_administracao',
 'tx_composta_sim',
 'aplica_outro_fundo',
 'tx_adm_maxima',
 'valor_saida',
 'tx_entrada_nao',
 'tx_performance_sim',
 'data_emissao_cota',
 'data_pagamento_resgate',
 'perfil_cota',
 'Formula Performance',
 'periodicidade_performance',
 'alavancado',
 'benchmark',
 'tx_saida_nao',
 'tx_performance_nao',
 'data_pagamento_conversao',
 'adm+cust',
 'perfil_tx_adm',
 'tx_adm_reais',
 'adm+gest+cust',
 'tributação',
 'previdenciario',
 'duração',
 'adm+gest',
 'infraestrutura',
 'composicao_s',
 'taxa_gestão',
 'investe_tx_0',
 'PGBL',
 'VGBL']


In [39]:
for ent in entidades:
    ner.add_label(ent)

In [40]:
def train_new_ner (data, n_interations, model=None):
    TRAIN_DATA = data
    dir_model = "../models/"
    
    
    if 'ner' not in nlp_spacy.pipe_names:
        ner = nlp_spacy.create_pipe('ner')
        nlp_spacy.add_pipe(ner, last=True)
    else: 
        ner = nlp_spacy.get_pipe('ner')
    
    #adicionando labels
    
    for _, annotations in TRAIN_DATA: 
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])
            
    #other pipes para desativar durante o train do novo ner 
    
    other_pipes = [pipe for pipe in nlp_spacy.pipe_names if pipe!= 'ner']
    with nlp_spacy.disable_pipes(*other_pipes): #desativa os outros pipe tagger, parser e etc
        optimizer = nlp_spacy.begin_training() #treinando o modelo 
        
        for inter in tqdm(range(n_interations)):
            random.shuffle(TRAIN_DATA)
            losses ={}
            
            
            for text, annotations in TRAIN_DATA:
                try:

                    nlp_spacy.update(
                        [text], #textos  
                        [annotations], #anotações 
                        drop=0.2,  
                        sgd=optimizer, #atualizar pesos 
                        losses=losses) 
                except Exception as error:
                    pass
            print(losses)
    return nlp_spacy 

In [42]:
model_ner = train_new_ner(TRAIN_DATA, 80)

  0%|          | 0/80 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "REGULAMENTO DO [DETERMINAR NOME] (47798) 1 REGULAM..." with entities "[[8174, 8182, 'alavancado'], [7232, 7242, 'investi...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\ProgramData\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "FUNDO DE INVESTIMENTO EM AÇÕES SABESPREV DIVIDENDO..." with entities "[[7086, 7098, 'percentual_credito'], [42939, 42942...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\ProgramData\Anaconda3\lib\site-packages\spacy\language.py:482: User

{'ner': 382180.8425049782}


  2%|▎         | 2/80 [05:27<3:35:18, 165.62s/it]

{'ner': 411320.5312371254}


  4%|▍         | 3/80 [08:26<3:37:33, 169.53s/it]

...

{'ner': 383610.0927145481}


 36%|███▋      | 29/80 [1:22:33<3:06:22, 219.26s/it]

{'ner': 373102.4840044975}


 36%|███▋      | 29/80 [1:26:10<2:31:33, 178.31s/it]


KeyboardInterrupt: 